In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://kleinpennyrentals.com/list"
propertyListingPage = requests.get(url)
propertyListingSoup = BeautifulSoup(propertyListingPage.content, "html.parser")

In [3]:
propertyLinks = []

# Getting all properties in the main listing page
properties = propertyListingSoup.find_all("td", {"class": "views-field-title"})

baseURL = "https://kleinpennyrentals.com"
# Looping through to get each property link
for property in properties:
  link = property.find_all("a")[0]["href"]
  propertyLinks.append(baseURL + link)

In [50]:
propertyLinks

['https://kleinpennyrentals.com/athens-ohio-housing/12-n-congress-st-apt-7',
 'https://kleinpennyrentals.com/athens-ohio-housing/12-n-congress-st-apt-8',
 'https://kleinpennyrentals.com/athens-ohio-housing/2-foster-pl-apt-a',
 'https://kleinpennyrentals.com/athens-ohio-housing/33-e-carpenter-st-apt-a',
 'https://kleinpennyrentals.com/athens-ohio-housing/33-e-carpenter-st-apt-c',
 'https://kleinpennyrentals.com/athens-ohio-housing/36-12-w-state-st',
 'https://kleinpennyrentals.com/athens-ohio-housing/38-w-state-st-apt-b',
 'https://kleinpennyrentals.com/athens-ohio-housing/47-w-washington-st-apt-1',
 'https://kleinpennyrentals.com/athens-ohio-housing/47-w-washington-st-apt-2',
 'https://kleinpennyrentals.com/athens-ohio-housing/47-w-washington-st-apt-3',
 'https://kleinpennyrentals.com/athens-ohio-housing/47-w-washington-st-apt-4',
 'https://kleinpennyrentals.com/athens-ohio-housing/47-w-washington-st-apt-5',
 'https://kleinpennyrentals.com/athens-ohio-housing/47-w-washington-st-apt-6',

In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

 ········


In [5]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=propertyLinks
)

In [7]:
docs = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [9]:
import getpass
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()  # Update to your API key

 ········


In [10]:
from langsmith import Client

client = Client()


In [11]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

In [12]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [29]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [30]:
rag_chain.invoke("What is a good 2 bedroom apartment that is cheaper than 1200 a month")

/Users/loganrichards/Desktop/KleinPennySearch/KleinPennyRAGBot/myenv/lib/python3.9/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=5b542fbd-e8d5-4b8d-9b38-744fcdc53aff,id=5b542fbd-e8d5-4b8d-9b38-744fcdc53aff; trace=5b542fbd-e8d5-4b8d-9b38-744fcdc53aff,id=9eb4adca-1119-43ee-8f59-ce7545c32668; trace=5b542fbd-e8d5-4b8d-9b38-744fcdc53aff,id=4b705943-93ff-4481-bdf0-a330553414a3; trace=5b542fbd-e8d5-4b8d-9b38-744fcdc53aff,id=32c0a7f3-b0cc-4c94-89af-7e619193bee4; trace=5b542fbd-e8d5-4b8d-9b38-744fcdc53aff,id=d43a2f48-e64e-4cd2-820b-10233dd1c20f
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authenticatio

'A good option for a 2-bedroom apartment under $1200 a month is at 74 E State St, Apt 12, priced at $566 per bedroom, totaling $1132. This apartment features amenities like in-unit laundry, central air conditioning, and a convenient location near uptown Athens. Another option is 38 W State St, Apt A, also priced at $566 per bedroom, offering similar amenities and a prime location close to the action.'

Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=5b542fbd-e8d5-4b8d-9b38-744fcdc53aff,id=1a633506-9164-4585-9245-fefa00c60ca1; trace=5b542fbd-e8d5-4b8d-9b38-744fcdc53aff,id=61c2d316-d4d9-4cc4-b542-e0b27014473f; trace=5b542fbd-e8d5-4b8d-9b38-744fcdc53aff,id=5b542fbd-e8d5-4b8d-9b38-744fcdc53aff
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=1eb98a92-6c6f-471c-92cf-0f37dcfde450,id=1eb98a92-6c6f-471c-92cf-0f37dcfde450; trace=1eb98a92-6c6f-471c-92cf-0f37dcfde450,id=038f7ce9-2701-4776-8f29-bb9880c97cda; trace=1eb98a92-6c6f-471c-92cf-0f37dcf

In [31]:
# cleanup
vectorstore.delete_collection()

In [46]:
from langchain_core.prompts import PromptTemplate

In [56]:
template = """Use the following pieces of context to answer the question given by a potential tenant about properties.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum, keep the answer as concise as possible, and provide the link to each property found from the source in the metadata.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [59]:
from langchain_core.runnables import RunnableLambda

def inspect(state):
    """Print the state passed between Runnables in a langchain and pass it on"""
    print(state)
    return state
    
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | RunnableLambda(inspect)
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [60]:
rag_chain.invoke("What are some 2 bedroom properties with one bathroom cheaper than 1200")

{'context': '74 E State, Apt 6: 2 Beds, 1 Block to Uptown, New Bathroom  Skip to main content  info@kleinpennyrentals.com740-594-6900      Main navigation    Housing   Detailed Search   Property List   Utilities   FAQ   Sample Lease   About Us   Contact Us   KP Educational Fund   Blog                                                 PreviousNext 74 E State St Apt 621.0  2Max occupancy2AvailableMay 2026Price$566 per bedroom per monthLeaseMay to MayIncludedWaterSewageGarbageNot includedElectricInternetAmenitiesAir conditioningBalconyCeramic tiled bathDishwasherMicrowaveNew bathroomNew carpetNew ceramic tile kitchen floorParking AvailableWasher and dryerWe just redecorated this cool 2-bedroom apartment at our 74 E State rental building, painting the walls a dramatic red and the trim a crisp white. The carpet is new in the living room and adds to the overall cozy ambiance of this open floorplan. We also remodeled the bathroom. A super-efficient mini-split system provides heat and AC.\xa0Own

'You can consider the following two-bedroom properties, each priced at $566 per bedroom per month, totaling $1132: \n\n1. [74 E State, Apt 6](https://kleinpennyrentals.com/74-e-state-apt-6) - Recently remodeled with new carpet and bathroom. \n\n2. [74 E State St, Apt 12](https://kleinpennyrentals.com/74-e-state-apt-12) - Features central air, a spacious kitchen, and in-unit laundry. \n\nThanks for asking!'

TypeError: string indices must be integers

In [55]:
docs[0]

Document(metadata={'source': 'https://kleinpennyrentals.com/athens-ohio-housing/12-n-congress-st-apt-7', 'title': '12 N Congress, Apt 7: 1 Bed Ohio University Rental in Athens', 'description': 'Featuring a completely redone kitchen and bathroom, this 1 bedroom apartment for rent is super close to campus and even closer to uptown Athens bars and shops.', 'language': 'en'}, page_content='12 N Congress, Apt 7: 1 Bed Ohio University Rental in Athens  Skip to main content  info@kleinpennyrentals.com740-594-6900      Main navigation    Housing   Detailed Search   Property List   Utilities   FAQ   Sample Lease   About Us   Contact Us   KP Educational Fund   Blog                                                       PreviousNext 12 N Congress St Apt 711.0  1–2Max occupancy2AvailableMay 2026Price$566 per bedroom per monthLeaseMay to MayIncludedWaterSewageGarbageNot includedElectricGasInternetAmenitiesCeramic tiled bathLarge closetLaundry roomNew bathroomThis\xa0one bedroom off-campus apartment 

In [65]:
response = rag_chain.invoke("What are some 2 bedroom properties with one bathroom cheaper than 1200")

{'context': '74 E State, Apt 6: 2 Beds, 1 Block to Uptown, New Bathroom  Skip to main content  info@kleinpennyrentals.com740-594-6900      Main navigation    Housing   Detailed Search   Property List   Utilities   FAQ   Sample Lease   About Us   Contact Us   KP Educational Fund   Blog                                                 PreviousNext 74 E State St Apt 621.0  2Max occupancy2AvailableMay 2026Price$566 per bedroom per monthLeaseMay to MayIncludedWaterSewageGarbageNot includedElectricInternetAmenitiesAir conditioningBalconyCeramic tiled bathDishwasherMicrowaveNew bathroomNew carpetNew ceramic tile kitchen floorParking AvailableWasher and dryerWe just redecorated this cool 2-bedroom apartment at our 74 E State rental building, painting the walls a dramatic red and the trim a crisp white. The carpet is new in the living room and adds to the overall cozy ambiance of this open floorplan. We also remodeled the bathroom. A super-efficient mini-split system provides heat and AC.\xa0Own

In [13]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are given a list of documents (each associated with a property) and their associated descriptions, titles, and other information as context. "
    "Use the following pieces of retrieved context to give properties that match the user's question."
    "If you don't know the answer, say that you "
    "don't know. If you use any information from the context, list its metadata in your answer." 
    ""
    "\n\n"
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [16]:
response = rag_chain.invoke({"input": "Help me find an apartment with a cast-iron and porcelain Kohler tub"})

In [85]:
response["context"][1]

Document(metadata={'description': 'Featuring a completely redone kitchen and bathroom, this 1 bedroom apartment for rent is super close to campus and even closer to uptown Athens bars and shops.', 'language': 'en', 'source': 'https://kleinpennyrentals.com/athens-ohio-housing/12-n-congress-st-apt-7', 'title': '12 N Congress, Apt 7: 1 Bed Ohio University Rental in Athens'}, page_content='12 N Congress, Apt 7: 1 Bed Ohio University Rental in Athens  Skip to main content  info@kleinpennyrentals.com740-594-6900      Main navigation    Housing   Detailed Search   Property List   Utilities   FAQ   Sample Lease   About Us   Contact Us   KP Educational Fund   Blog                                                       PreviousNext 12 N Congress St Apt 711.0  1–2Max occupancy2AvailableMay 2026Price$566 per bedroom per monthLeaseMay to MayIncludedWaterSewageGarbageNot includedElectricGasInternetAmenitiesCeramic tiled bathLarge closetLaundry roomNew bathroomThis\xa0one bedroom off-campus apartment 

In [17]:
print(response["answer"])
# response["context"][0]
print("\n\nYou can check out more related properties here: ")
for found in response["context"]:
    print(found.metadata["title"] + ": " + found.metadata["source"])

I found a few apartments that feature a Kohler cast iron porcelain tub:

1. **20 N Congress St Apt 2**: 
   - **Bedrooms**: 7
   - **Bathrooms**: 7
   - **Amenities**: Each bedroom has a private bathroom with a Kohler cast iron porcelain tub.
   - **Available**: May 2026
   - **Price**: $700 per bedroom per month
   - **Lease**: May to May
   - **Security Deposit**: $700 per person
   - **Contact**: info@kleinpennyrentals.com, 740-594-6900

2. **20 N Congress St Apt 5**: 
   - **Bedrooms**: 4
   - **Bathrooms**: 4
   - **Amenities**: Each bedroom has its own private bath featuring a Kohler cast iron porcelain tub.
   - **Available**: May 2026
   - **Price**: $700 per bedroom per month
   - **Lease**: May to May
   - **Security Deposit**: $700 per person
   - **Contact**: info@kleinpennyrentals.com, 740-594-6900

3. **20 N Congress St Apt 4**: 
   - **Bedrooms**: 4
   - **Bathrooms**: 4
   - **Amenities**: Each bedroom has a private bathroom with a Kohler cast iron porcelain tub.
   - *